In [68]:
from  tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras import regularizers
from joblib import load
import datetime
import numpy as np
import pickle

model = load_model("model.h5")

scaler = StandardScaler()
input_data = {
    "CreditScore": [600,700,900],
    "Geography": ["France","Germany","Spain"],
    "Gender": ["Male","Female","Female"],
    "Age": [40,60,80],
    "Tenure": [3,8,10],
    "Balance": [60000,80000,95000],
    "NumOfProducts": [2,5,3],
    "HasCrCard": [1,1,0],
    "IsActiveMember": [1,0,1],
    "EstimatedSalary": [50000,65000,80000],
    "Exited":[1,0,1]
}


## label encoding for gender

# input_df = pd.DataFrame({k: [v] for k, v in input_data.items()})
input_df = pd.DataFrame(input_data)

label_Encoded= LabelEncoder()
label_Encoded.fit(input_df ["Gender"])
oneHot_encoder_geo = OneHotEncoder(sparse_output=False)
oneHot_encoder_geo.fit(input_df [["Geography"]])

# save file as pickle

# with open("gender_encoder.pkl", "wb") as f:
#     pickle.dump(label_Encoded, f)

# with open("geo_encoder.pkl", "wb") as f:
#     pickle.dump(oneHot_encoder_geo, f)

with open("gender_encoder.pkl", "rb") as f:
    gender_encoder = pickle.load(f)

with open("geo_encoder.pkl", "rb") as f:
    geo_encoder = pickle.load(f)


input_df["Gender_encoded"] = gender_encoder.transform(input_df["Gender"])
input_df = input_df.drop("Gender", axis=1)

## oneHot encoder
getEncoded_geo = geo_encoder.transform(input_df[["Geography"]])
encoded_geo_df = pd.DataFrame(getEncoded_geo, columns=geo_encoder.get_feature_names_out(["Geography"]))
input_df = pd.concat([input_df.drop("Geography", axis=1).reset_index(drop=True),
                      encoded_geo_df.reset_index(drop=True)], axis=1)
input_df = input_df.drop("Exited", axis=1)

if "Exited" in input_df.columns:
    input_df = input_df.drop("Exited", axis=1)

scaled_input = scaler.fit_transform(input_df)
scaled_df = pd.DataFrame( input_df, columns = input_df.columns)
predictions = model.predict(scaled_input)
predicted_classes = (predictions > 0.5).astype(int)
predicted_classes
scaled_df



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_encoded,Geography_France,Geography_Germany,Geography_Spain
0,600,40,3,60000,2,1,1,50000,1,1.0,0.0,0.0
1,700,60,8,80000,5,1,0,65000,0,0.0,1.0,0.0
2,900,80,10,95000,3,0,1,80000,0,0.0,0.0,1.0


In [73]:
prdiction_prob = predictions[0][0]
prdiction_prob

np.float32(0.512175)

In [72]:
if prdiction_prob >0.5:
    print("The customer is likely to churn")
else:
     print("The customer is not likely to churn")

The customer is likely to churn


0    1
1    2
2    3
dtype: int64